# Mapas Interactivos

Como ultimo ejemplo del uso de mapas para la visualización de datos geoespaciales, veremos la utilización de la librería folium para la representación de los mismos.

Las ventajas de los mapas interactivos son similares a las que hemos visto para visualizaciones estadísticas: en un entrono digital con posibilidad de interacción, el usuario puede encontrarlos más útiles, al poder filtrar datos y quedarse con los más interesantes para él. Como contraprestación, se necesitan de más recursos para funcionar. 

Es recomendable plantearse si es realmente necesaria esta interacción.

In [17]:
# Como siempre, si no contamos con la librería 'folium', antes de nada, necesitamos instalarla en el sistema:
# ! pip install folium
# ! anaconda install folium

In [18]:
# Importamos algunos de los tipos de mapas a emplear
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster

import numpy as np
import math

In [19]:
# En caso de que trabajemos en el entorno de Google, podemos acceder a los datos guardados ahí
#from google.colab import drive
#drive.mount('/content/drive')

In [34]:
#data_dir = '/content/drive/My Drive/VisualizDatos/GeoEspacial/geospatial-learn-course-data/'
data_dir = '../data/geoespacial/crimes-in-boston/'

Dependiendo del entorno donde nos encontremos ejecutando, puede ser necesario embeber el objeto gráfico del mapa sobre un documento en html para poder mostrarlo después.

En caso de ser necesario, se puede definir una función `embed_map()` para mostrar mapas interactivos. Acepta dos argumentos: la variable que contiene el mapa y el nombre del archivo HTML donde se guardará el mapa. Esta función asegura que los mapas sean visibles en todos los navegadores web.

En Google Colab, no es necesario emplearlo, ya que es capaz de mostrar el mapa directamente.


In [21]:
# Function for displaying the map
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

La librería de `folium` trabaja accediendo a la API del conocido servicio de mapas *open source* [OpenStreetMaps](https://www.openstreetmap.org/). Este cede sus datos geográficos. 

Este servicio se encarga de agrupar y gestionar las aportaciones desinteresadas de millones de usuarios que comparten las características geográficas de sus lugares de residencia. Según el país al que accedamos, la información es más o menos completa. En España casi todos los lugares cuentan con información actualizada.

A su vez, se emplea la librería javascript [Leaflet](https://leafletjs.com/) que permite el dibujo de esos datos sobre una página web en forma de mapa interactivo.

Para obtener un mapa geográfico estándar, con los controles de zoom a los que estamos habituados, basta con una llamada al objeto `Map`, indicando algunos parámetros:

In [22]:
# Create a map
m_1 = folium.Map(location=[42.32,-71.0589], tiles='openstreetmap', zoom_start=10)

# En google Colab, no es necesario emplear el emmbeding
# Display the map
#embed_map(m_1, 'm_1.html')
m_1

Varios argumentos nos permiten personalizar la apariencia del mapa:

* location establece el centro inicial del mapa. Utilizamos la latitud (42.32° N) y longitud (-71.0589° E) de la ciudad de Boston, en EEUU.
* cambia el estilo del mapa; en este caso, elegimos el estilo OpenStreetMap. Si teneis curiosidad, podeís encontrar otras opciones listadas [aquí](https://github.com/python-visualization/folium/tree/master/folium/templates/tiles).
* zoom_start establece el nivel inicial de zoom del mapa, donde los valores más altos se acercan más al mapa.

Se puede probar a explorar acercándose y alejándose, o arrastrando el mapa en diferentes direcciones.

##Añadir datos sobre el mapa

La ventaja de esta librería es que resulta relativamente sencillo realizar representaciones con marcadores sobre los estos mapas.  
Para ello, se pueden cargar los datos a representar a un Dataframe de `pandas` y a continuación asociarlos a un mapa ya creado (siempre que cada muestra incluya información geográfica).

### Lectura de datos
En este ejemplo, trabajaremos zonas donde se han registrado hechos delictivos en al ciudad de Boston:

In [35]:
import pandas as pd

crimes = pd.read_csv(data_dir+'/crimes-in-boston/crime.csv', 
                     sep=',', encoding='cp1252')
crimes.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,I182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,NaN,2018-09-02 13:00:00,2018,9,Sunday,13,Part One,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)"
1,I182070943,1402,Vandalism,VANDALISM,C11,347,NaN,2018-08-21 00:00:00,2018,8,Tuesday,0,Part Two,HECLA ST,42.306821,-71.060300,"(42.30682138, -71.06030035)"
2,I182070941,3410,Towed,TOWED MOTOR VEHICLE,D4,151,NaN,2018-09-03 19:27:00,2018,9,Monday,19,Part Three,CAZENOVE ST,42.346589,-71.072429,"(42.34658879, -71.07242943)"
3,I182070940,3114,Investigate Property,INVESTIGATE PROPERTY,D4,272,NaN,2018-09-03 21:16:00,2018,9,Monday,21,Part Three,NEWCOMB ST,42.334182,-71.078664,"(42.33418175, -71.07866441)"
4,I182070938,3114,Investigate Property,INVESTIGATE PROPERTY,B3,421,NaN,2018-09-03 21:05:00,2018,9,Monday,21,Part Three,DELHI ST,42.275365,-71.090361,"(42.27536542, -71.09036101)"


In [36]:
crimes.describe()

,OFFENSE_CODE,YEAR,MONTH,HOUR,Lat,Long
count,319073.000000,319073.000000,319073.000000,319073.000000,299074.000000,299074.000000
mean,2317.546956,2016.560586,6.609719,13.118205,42.214381,-70.908272
std,1185.285543,0.996344,3.273691,6.294205,2.159766,3.493618
min,111.000000,2015.000000,1.000000,0.000000,-1.000000,-71.178674
25%,1001.000000,2016.000000,4.000000,9.000000,42.297442,-71.097135
50%,2907.000000,2017.000000,7.000000,14.000000,42.325538,-71.077524
75%,3201.000000,2017.000000,9.000000,18.000000,42.348624,-71.062467
max,3831.000000,2018.000000,12.000000,23.000000,42.395042,-1.000000


Nótese que los datos incluyen los campos `STREET`, `Lat`, `Long`, `Location` que permiten establecer el lugar físico de cada uno.


### Representando puntos

Para reducir la cantidad de datos que necesitamos incluir en el mapa, limitaremos (temporalmente) nuestra atención a los robos diurnos. (`Robbery` en horas entre 9 y 18)

In [37]:
daytime_robberies = crimes[((crimes.OFFENSE_CODE_GROUP == 'Robbery') & \
                            (crimes.HOUR.isin(range(9,18))))]

#daytime_robberies = daytime_robberies.dropna(how='any')
daytime_robberies = daytime_robberies[np.isfinite(daytime_robberies['Lat'])]

daytime_robberies.describe()

,OFFENSE_CODE,YEAR,MONTH,HOUR,Lat,Long
count,1413.000000,1413.000000,1413.000000,1413.000000,1413.000000,1413.000000
mean,321.203822,2016.414720,6.828733,13.731069,42.290632,-71.030243
std,26.981860,1.003173,3.485558,2.453218,1.152845,1.864487
min,301.000000,2015.000000,1.000000,9.000000,-1.000000,-71.172440
25%,301.000000,2016.000000,4.000000,12.000000,42.300144,-71.091043
50%,301.000000,2016.000000,7.000000,14.000000,42.322770,-71.076245
75%,351.000000,2017.000000,10.000000,16.000000,42.344396,-71.062899
max,381.000000,2018.000000,12.000000,17.000000,42.384902,-1.000000


Para añadir puntos indivuduales sobre el mapa, empleamos la clase `Marker`. Permite añadir un punto de marca, indicando latitud y longitud.

In [38]:
# Crear el mapa
m_2 = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=13)

# Añadimos los puntos sobre el mapa, iterando sobre el Dataframe
for idx, row in daytime_robberies.iterrows():
    Marker([row['Lat'], row['Long']]).add_to(m_2) #Se genera el marcador y se añade al mapa

# Mostramos el mapa
m_2

### Representando Grupos

Si tenemos muchos marcadores que añadir, `folium.plugins.MarkerCluster()` puede ayudar a despejar el mapa. Cada marcador se añade a un objeto `MarkerCluster`.

In [39]:
# Se crea el mapa
m_3 = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=13)

# Se añaden los puntos al mapa
mc = MarkerCluster() # Se genera el agrupador
for idx, row in daytime_robberies.iterrows():
    if not math.isnan(row['Long']) and not math.isnan(row['Lat']):
        mc.add_child(Marker([row['Lat'], row['Long']])) # Se añaden al agrupador
m_3.add_child(mc)

# Se representa el mapa
#embed_map(m_3, 'm_3.html')
m_3

Si hacemos zoom sobre el mapa, veremos como se desagregan los datos de los conjuntos sobre los que nos vayamos acercando.

## Mapas de Burbujas

Un mapa de burbujas utiliza círculos en lugar de marcadores. Variando el tamaño y el color de cada círculo, también podemos mostrar la relación entre la ubicación y otras dos variables (una empleando color y otra empleando tamaño).

Al igual que antes, se crea un mapa de burbujas usando `folium.Circle()` para añadir círculos de forma iterativa. En la celda de código de abajo, los robos que ocurrieron en las horas 9-12 se representan en verde, mientras que los robos de las horas 13-17 se representan en rojo.

In [40]:
# Se crea el mapa
m_4 = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=13)

# Función que asigna el color dependiendo de la hora
def color_producer(val):
    if val <= 12:
        return 'forestgreen'
    else:
        return 'darkred'

# Se añaden los círculos 1 a 1 al mapa, de forma similar a los marcadores
for i in range(0,len(daytime_robberies)):
    Circle(
        location=[daytime_robberies.iloc[i]['Lat'], daytime_robberies.iloc[i]['Long']],
        radius=20,
        color=color_producer(daytime_robberies.iloc[i]['HOUR'])).add_to(m_4)

# Display the map
# embed_map(m_4, 'm_4.html')
m_4

A la vista de los datos, no se puede decir que haya un patrón claro en cuanto a los horarios de los incidentes: no se aprecia una zona clara en la que se den muchos mas incidente por la mañana que por la tarde.

## Mapas de Calor

Para crear un mapa de calor, usamos `folium.plugins.HeatMap()`. Esto muestra la densidad de la delincuencia en diferentes áreas de la ciudad, donde las áreas rojas tienen relativamente más incidentes delictivos.

Como era de esperar en una gran ciudad, la mayoría de los crímenes ocurren cerca del centro.

In [41]:
# Creamos el mapa
m_5 = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=12)


# Añadimos la capa de HeatMap sobre el mapa inicial
crimes_pos = pd.DataFrame(index=crimes.index)
crimes_pos['Lat'] = pd.to_numeric(crimes['Lat'])
crimes_pos['Long'] = pd.to_numeric(crimes['Long'])

HeatMap(data=crimes_pos, radius=10).add_to(m_5)

# Mostramos el mapa
#embed_map(m_5, 'm_5.html')
m_5

ValueError: Location values cannot contain NaNs.

## Mapas Cloropéticos

La librería también permite la construcción de este tipo de mapas, coloreados en función de una variable. 

En el ejemplo, primero se intentará asociar cada distrito de la ciudad con el número de crímenes en cada uno (como ya sabemos, con el método `value_counts`). Ésta información ya está contenida en nuestro DF sobre crimenes.

Luego los representaremos sobre el mapa: obtendremos esta información de otro fichero en el que aparecen los límites de los distritos.

In [42]:
# Numero de crimenes por cada distrito
plot_dict = crimes.DISTRICT.value_counts()
plot_dict.head()

B2     49945
C11    42530
D4     41915
A1     35717
B3     35442
Name: DISTRICT, dtype: int64

In [43]:
#! pip install geopandas

In [44]:
import geopandas as gpd

In [46]:
# GeoDataFrame con las divisiones de los distritos policiales de Boston
districts_full = gpd.read_file(data_dir+'/Police_Districts/Police_Districts/Police_Districts.shp')
districts = districts_full[["DISTRICT", "geometry"]].set_index("DISTRICT")
districts.head()

,geometry
DISTRICT,
A15,(POLYGON ((-71.07415718153364 42.3905076862483...
A7,(POLYGON ((-70.99644430907341 42.3955679826137...
A1,POLYGON ((-71.05199523849357 42.36883599550553...
C6,POLYGON ((-71.04405776717314 42.35403006334784...
D4,"POLYGON ((-71.07416484856725 42.3572379188053,..."


Es muy importante que tanto los datos que se representan por colores (en este caso `plot_dict`), como los datos que marcan la componente geográfica (en este ejemplo son polígonos y están en `districts`), tengan exactamente el mismo índice para poder relacionarlos en el mapa.

In [47]:
# Se crea el mapa inicial
m_6 = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=12)

# Se crea el objeto del mapa cloroplético y se asocia al mapa
Choropleth(geo_data=districts.__geo_interface__, 
           data=plot_dict, 
           key_on="feature.id", 
           fill_color='YlGnBu', 
           legend_name='Major criminal incidents (Jan-Aug 2018)'
          ).add_to(m_6)

# Display the map
#embed_map(m_6, 'm_6.html')
m_6

`folium.Choropleth()` toma varios argumentos:

* `geo_data`: es una "GeoJSON FeatureCollection" que contiene los límites de cada área geográfica.
  * En el código anterior, convertimos el GeoDataFrame de los distritos en una GeoJSON FeatureCollection con el atributo `__geo_interface__`.
* `data`: es una serie de Pandas que contiene los valores que se utilizarán para codificar por color cada área geográfica.
* `key_on` siempre se asignará el valor `feature.id`.
  * Esto se refiere al hecho de que el GeoDataFrame utilizado para geo_datos y la serie Pandas proporcionada en los datos tienen el mismo índice. Para entender los detalles, tendríamos que mirar más de cerca la estructura de una colección de características de GeoJSON (donde el valor correspondiente a la clave "características" es una lista, donde cada entrada es un diccionario que contiene una clave "id").
* `fill_color`: establece la escala de color.
* `legend_name`: etiqueta la leyenda en la esquina superior derecha del mapa.


## Conclusiones

La librería `folium` permite una gran variedad de mapas a representar, utilizanod como vemos unos pocos comandos simples. 

Se puede encontrar más documentación en su [página oficial](https://python-visualization.github.io/folium/).

Como sucedía con otras librerías más avanzadas (como `plotly`) se debe considerar que se trata de librerías actualmente en desarrollo y menos establecidas que otras más simples y con más tiempo en desarrollo. Por ello, a pesar de obtener resultados muy vistosos con poco trabajo, se debe considerar que no existe tanta documentación y ejemplos y que es posible encontrar bugs en su funcionamiento.